---
toc: true
comments: false
layout: post
title: Finance Math Blog
description: Python Invesment Portfolio Simulator
courses: { compsci: {week: 2} }
type: hacks
---

This blog takes 8 stocks and simulates how well a portfolio would do if invested. It also tells what the best weighting for each stock would be the most optimal. It gives important information such as risk, volatility, profit, Sharpe Ratio, etc. 

8 stocks have been chosen at random which are Intel (INTC), Ford Motor Company (F), Walt Disney Co (DIS), Tesla (TSLA), Amazon (AMZN), Bank of America (BAC), Sony (SONY), and Meta (META)

In [10]:
# Reading in the stocks of each stock and then creating a central data frame of each. 

import numpy as np
import pandas as pd
import pandas_datareader.data as web
# Get stock data  
all_data = {ticker: web.DataReader(ticker,'stooq')
           for ticker in ['INTC', 'F', 'DIS', 'TSLA', 'AMZN', 'BAC', 'SONY', 'META']}
# Extract the 'Adjusted Closing Price'
price = pd.DataFrame({ticker: data['Close']
                     for ticker, data in all_data.items() })

price

,INTC,F,DIS,TSLA,AMZN,BAC,SONY,META
Date,,,,,,,,
2023-09-01,36.6100,12.14000,81.64,245.0100,138.1200,28.9800,85.2600,296.38
2023-08-31,35.1400,12.13000,83.68,258.0800,138.0100,28.6700,83.1900,295.89
2023-08-30,34.5300,12.03000,84.28,256.9000,135.0700,29.0400,82.3500,295.10
2023-08-29,34.3100,12.05000,84.40,257.1800,134.9100,29.1700,82.6100,297.99
2023-08-28,33.6200,11.90000,84.16,238.8200,133.1400,28.7600,82.0700,290.26
...,...,...,...,...,...,...,...,...
2018-09-10,41.0554,8.18682,108.50,19.0333,96.9505,27.9808,55.6325,164.18
2018-09-07,41.1893,8.09099,108.79,17.5493,97.6035,28.0181,55.4838,163.04
2018-09-06,41.9072,8.23033,108.09,18.7300,97.9155,28.0093,55.3192,162.53


The standard deviation is how investors measure volatility and risk. STD measures the average distance from the mean, so the higher the STD the less concise the data is. In the finance world, if the STD is higher, that means that the price of the stock can be more unpredictable. This equates to risk. The lower the STD the better, and vice versa. 

In [3]:
# finding standard deviation
price.std()

INTC      9.261005
F         3.719340
DELL     11.215586
TSLA    112.395614
AMZN     32.606867
BAC       6.758436
SONY     21.724307
LYFT     17.977489
dtype: float64

The correlation between each of the stocks shows how well the portfolio will do. The correlation in math shows the relationship and the proportion between two variables. In finance the correlation dictates how two stocks will react in relationship. In layman terms it shows if one stock will go up what will another stock do. Correlation ranges from -1 to 1, where -1 is the most optimal. 

In [4]:
# finding correlation of stocks
price.corr()

,INTC,F,DELL,TSLA,AMZN,BAC,SONY,LYFT
INTC,1.000000,-0.206477,-0.196873,-0.136137,0.375671,0.087358,0.103368,0.763920
F,-0.206477,1.000000,0.876058,0.827536,0.448809,0.880314,0.749404,-0.082744
DELL,-0.196873,0.876058,1.000000,0.876392,0.596801,0.794300,0.831929,-0.133084
TSLA,-0.136137,0.827536,0.876392,1.000000,0.724181,0.745392,0.887772,-0.213905
AMZN,0.375671,0.448809,0.596801,0.724181,1.000000,0.523433,0.806234,0.271340
BAC,0.087358,0.880314,0.794300,0.745392,0.523433,1.000000,0.763487,0.189696
SONY,0.103368,0.749404,0.831929,0.887772,0.806234,0.763487,1.000000,0.012364
LYFT,0.763920,-0.082744,-0.133084,-0.213905,0.271340,0.189696,0.012364,1.000000


By finding the correlation of the entire portfolio, it will show how well it will do. The closer to -1 the better. 

In [5]:
# Finding average correlation to show profability of portfolio
averageCorr = price.corr()
averageCorrMean = averageCorr.mean()
averageCorrMean

column_sum = 0

# For loop to find the mean of the entire data table
for i in range(len(averageCorrMean)):
    column_sum += averageCorrMean[i]
column_sum = column_sum/len(averageCorrMean)
column_sum

/tmp/ipykernel_245673/2458078143.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  column_sum += averageCorrMean[i]


0.5052043991861102

This is where the math and the real fun begins. This next code finds the optimal weights of each stock. It does this by running through 6000 differnt scenarios each with different weighting. It finds the weights by finding the retention factor and the volatility of each stock and then compare it to each of the other 7 stocks. Then once it has done that it'll compare it to average yearly stock prices where it will then compare all 6000 scenarios and output the most optimal. 

In [11]:
# finding weights, return, volitilty, and sharpe ratio. 

stocks = pd.concat([price['INTC'], price['F'], price['DIS'], price['TSLA'], price['AMZN'], price['BAC'], price['SONY'], price['META']], axis = 1)
log_ret = np.log(stocks/stocks.shift(1))

# setting up variables
np.random.seed(42)
num_ports = 6000
num_stocks = 8
all_weights = np.zeros((num_ports, len(stocks.columns)))
ret_arr = np.zeros(num_ports)
vol_arr = np.zeros(num_ports)
sharpe_arr = np.zeros(num_ports)

# going through all possible weights
for x in range(num_ports):
    # Weights
    weights = np.array(np.random.random(num_stocks))
    weights = weights/np.sum(weights)
    
    # Save weights
    all_weights[x,:] = weights
    
    # Expected return
    ret_arr[x] = np.sum( (log_ret.mean() * weights * 252))
    
    # Expected volatility
    vol_arr[x] = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov()*252, weights)))
    
    # Sharpe Ratio
    sharpe_arr[x] = ret_arr[x]/vol_arr[x]

The Sharpe Ratio is how investors determine the profability of a portfolio in a single number that can be compared to other portfolios. In finance, the Sharpe ratio measures the performance of an investment such as a security or portfolio compared to a risk-free asset, after adjusting for its risk. 

In [12]:
# printing the max sharpe ratio
print("Max Sharpe Ratio = ",sharpe_arr.max())
sharpe_arr.argmax()
max_sr_ret =  ret_arr[sharpe_arr.argmax()]
max_sr_vol =  vol_arr[sharpe_arr.argmax()]

Max Sharpe Ratio =  0.005266972961310643
